![](https://straightpathsql.com/wp-content/uploads/2016/09/straight-path-solutions-logo-header.png)

# Workshop: PowerShell Automation for the DBA

#### _A Straight Path Consulting Course from David Seis_

![](https://raw.githubusercontent.com/microsoft/sqlworkshops/master/graphics/textbubble.png)

## Course Notebook: Module 3 - Advanced Automation with DBA Tools

Welcome to this workshop on \[\_PowerShell Automation for the DBA\_\]. In this Notebook, you'll apply the concepts you learned in this Module.

This Notebook contains recipes for some advanced powershell use cases. <span style="color: var(--vscode-foreground);">You'll need a working knowledge of Powershell Basics and Basic Powershell Automation.</span>

Please run the below script to configure your environment for these demos:

In [8]:
<# Environment Variables #>

$sqlinstance = 'SEIS-Work'
$outputPath = "C:\temp\Demo"
$CSVpath = "$outputpath\Home-lab_InstanceTracking.csv" 

If(!(test-path -PathType container $outputPath)) 
{ 
    New-Item -ItemType Directory -Path $outputPath 
    Write-Host "$outputPath Directory was created"
}

Write-Host "Environment configured"

Environment configured


## 3.1 Advanced PowerShell Automation: Files

## 

In the context of database administration, PowerShell’s versatility shines through its ability to save outputs in various relevant file formats. Tasks such as exporting data to CSV or generating reports in HTML format, which can be cumbersome when done manually, are made simple and efficient with PowerShell.

In [10]:
<# Code to create the Instance tracking csv #> 

clear-host


Write-Host "Instance tracking..." -ForegroundColor Green
$results =  Invoke-dbaquery -SqlInstance $SQLInstance -Query "
IF OBJECT_ID('tempdb..#ola') IS NOT NULL
DROP TABLE #ola
    CREATE TABLE #ola (
    DbName NVARCHAR(100)
    , Ola_Version nvarchar(10)
    )
        INSERT #ola
        EXEC sp_MSforeachdb '
        USE [?]
        SELECT DISTINCT db_name(), CASE WHEN CHARINDEX(N''--// Version: '', OBJECT_DEFINITION(obj.[object_id])) > 0 
        THEN CAST(LEFT(SUBSTRING(OBJECT_DEFINITION(obj.[object_id]),CHARINDEX(N''--// Version: '',OBJECT_DEFINITION(obj.[object_id])) + LEN(N''--// Version: '') + 1, 19),10) AS NVARCHAR(10)) END 
        FROM sys.objects AS obj
        INNER JOIN sys.schemas AS sch 
        ON obj.[schema_id] = sch.[schema_id]
        WHERE sch.[name] = N''dbo''
        AND obj.[name] IN (N''CommandExecute'')'


IF OBJECT_ID('tempdb..#FR') IS NOT NULL
DROP TABLE #FR
    CREATE TABLE #FR (
    dbName NVARCHAR(100)
    , FRDate nvarchar(10)
    )
        INSERT #FR
        EXEC sp_MSforeachdb '
        USE [?]
        SELECT DISTINCT db_name(), 
        CAST(CAST(create_date as DATE) as NVARCHAR(10))
        FROM sys.objects AS obj
        WHERE obj.[name] LIKE ''SP_Blitz'''

        IF EXISTS (SELECT * FROM sysobjects WHERE type = 'P' AND name = 'sp_Blitz')
        BEGIN
        DECLARE @VersionOutput VARCHAR(30), @VersionDateOutput DATETIME;
        EXEC sp_Blitz 
        @Version = @VersionOutput OUTPUT, 
        @VersionDate = @VersionDateOutput OUTPUT,
        @VersionCheckMode = 1;
        END 

        DECLARE @num_logs int;
        EXEC xp_instance_regread N'HKEY_LOCAL_MACHINE',N'Software\Microsoft\MSSQLServer\MSSQLServer',N'NumErrorLogs',@num_logs OUTPUT;


SELECT  
    SERVERPROPERTY('MachineName') AS Machine_Name
    ,   SERVERPROPERTY('ServerName') AS Instance_Name
    ,   CASE 
            WHEN Convert(nvarchar(20),SERVERPROPERTY('ProductVersion')) LIKE '16.%' THEN 2022
            WHEN Convert(nvarchar(20),SERVERPROPERTY('ProductVersion')) LIKE '15.%' THEN 2019
            WHEN Convert(nvarchar(20),SERVERPROPERTY('ProductVersion')) LIKE '14.%' THEN 2017
            WHEN Convert(nvarchar(20),SERVERPROPERTY('ProductVersion')) LIKE '13.%' THEN 2016
            WHEN Convert(nvarchar(20),SERVERPROPERTY('ProductVersion')) LIKE '12.%' THEN 2014
            WHEN Convert(nvarchar(20),SERVERPROPERTY('ProductVersion')) LIKE '11.%' THEN 2012
            WHEN Convert(nvarchar(20),SERVERPROPERTY('ProductVersion')) LIKE '10.%' THEN 2008
            ELSE NULL 
        END AS [SQL_Version]
    ,   SERVERPROPERTY('Edition') AS SQL_Edition
    ,   SERVERPROPERTY('ProductLevel') AS Product_Level
    ,   SERVERPROPERTY('ProductUpdateLevel') AS Update_Level
    ,   SERVERPROPERTY('ProductVersion') AS SQL_Build
    ,   (SELECT Ola_Version FROM #ola WHERE DbName in ('DB_Administration','Maintenance','DBA')) AS Ola_Version
    ,   (SELECT CAST(@VersionOutput as NVARCHAR(5)) FROM #FR WHERE dbName in ('master')) AS First_Responder_Version
    ,   (SELECT service_account FROM sys.dm_server_services WHERE servicename like 'SQL Server (%') as SQL_Account
    ,   (SELECT service_account FROM sys.dm_server_services WHERE servicename like 'SQL Server Agent%') as SQL_Agent_Account
    ,   (SELECT datediff(day, sqlserver_start_time, getdate()) FROM sys.dm_os_sys_info) as Uptime_Days
    ,   (SELECT is_disabled FROM sys.server_principals WHERE sid = 0x01 ) as is_sa_disabled
    ,   CONNECTIONPROPERTY('local_net_address') AS local_net_address
    ,   CONNECTIONPROPERTY('local_tcp_port') AS local_tcp_port
    ,   @num_logs as num_error_logs
    ,   GETDATE() AS Collection_Date

" 
$results |  Export-Csv -Path $CSVpath -Delimiter "," -Force -NoTypeInformation
$results 

Instance tracking...




Machine_Name            : Seis-WORK
Instance_Name           : Seis-WORK
SQL_Version             : 2022
SQL_Edition             : Developer Edition (64-bit)
Product_Level           : RTM
Update_Level            : 
SQL_Build               : 16.0.1115.1
Ola_Version             : 2022-12-03
First_Responder_Version : 8.13
SQL_Account             : NT Service\MSSQLSERVER
SQL_Agent_Account       : NT Service\SQLSERVERAGENT
Uptime_Days             : 0
is_sa_disabled          : False
local_net_address       : 
local_tcp_port          : 
num_error_logs          : 52
Collection_Date         : 5/4/2024 1:06:19 AM





## 3.2 Advanced PowerShell Automation: Emails

It can be used to send emails with reports derived from data queried from the database, or about the database itself. This can be a complex task when done manually, but PowerShell simplifies it, allowing for automated, timely, and consistent communication.

In [13]:
<# email integration #>
clear-host
<#required PSSENDGRID PowerShell module#>

$Client_Name = "Test Labs"
$Data_Collector = "David"

Write-Host "Security: Sending Test email..." -ForegroundColor Green
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
$Parameters = @{
    FromAddress     = "alertinbox@straightpathsql.com"
    ToAddress       = "david.seis@straightpathsql.com"
    Subject         = "$Client_Name Email Test"
    Body            = "$Data_Collector is testing email configuration on the $Client_Name environment."
    Token           = Get-Content -Path "$outputPath\Sendgrid.txt"
    FromName        = "$client_name"
    ToName          = "David"
}
Send-PSSendGridMail @Parameters 


Security: Sending Test email...


## 3.3 Advanced PowerShell Automation: Task Scheduling

The ability to create a script and then call them via tasks cheduler jobs is extermely useful, especially when you consider the versatility and range of Powershell, and th ebreadth of data you can access for these reports.

This script works but requires an environment where you have access to a service account that has log on as batch service as well as access to the targeted servers.

In [14]:
<# Create a report and task inside of taks sscheudler to check recent job runs #>
clear-host

    ## DEPENDENCIES
    # 1. Dbatools
    # 2. Service account acctess to all monitored servers

    <# Necessary Variables #>
    $reportname = "Job Run Report"

    $ClientName = 'Test-Lab' 
    $jobserver = 'Seis-Work'
    $monitoringTarget = $sqlinstance

    $ServiceAccount = $host.ui.PromptForCredential("Task Scheduler account", "Please enter the domainname\username and password for the service account that will run the tasks (password expiry = never is ideal). ", "DOMAIN\Account", "")

    $EmailRecipients = 'David.seis@straightpathsql.com;' #separate with semicolons

    Write-Host "Creating Powershell File..." -ForegroundColor Green
    New-Item -Path "\\$jobserver\C`$\StraightPath\Reports" -Name "$reportname`Script.ps1" -ItemType 'file' -value "
        
    <# Create C:\StraightPath\Reports for all the reports to go into#>
        `$dir = `"\\$jobserver\C$\StraightPath\Reports`"
        If(!(test-path -PathType container `$dir))
        {
            New-Item -ItemType Directory -Path `$path
        }
        `$path=`"\\$jobserver\C$\StraightPath\Reports\$clientname`_$reportname`$(get-date -f MM-dd-yyyy).htm`"

        `$SQLInstance= `"$monitoringTarget`"

    <#Define an Empty Array to Hold all of the HTML Fragments #>
        `$fragments = @()

    #Define the HTML style
        `$head = `"
            <style>
            body { background-color:#FAFAFA; font-family:Arial; font-size:12pt; }
            td, th { border:1px solid black; border-collapse:collapse; }
            th { color:white; background-color:black; }
            table, tr, td, th { padding: 2px; margin: 0px }
            tr:nth-child(odd) {background-color: lightgray}
            table { margin-left:50px; }
            </style>

            <H2>Daily Job Run report</H2>
            <H3>Automated powershell script from Straight Path IT Solutions, LLC.</H3>
            `"


        `$results = Invoke-DbaQuery -SQLInstance `$SQLInstance -Query `"
        WITH jobhistory as (   
            SELECT    job_id,
                        Max(instance_id) instance_id
            FROM      msdb.dbo.sysjobhistory
            WHERE     step_id = 0
                        AND run_status is not null
                        
            GROUP BY job_id
            --   order by run_status asc
            )
            SELECT  --  j.job_id,
                    j.name,
                    CASE sjh.run_status
                WHEN 0 THEN 'Failed'
                WHEN 1 THEN 'Succeeded'
                WHEN 2 THEN 'Retry'
                WHEN 3 THEN 'Canceled'
                WHEN 4 THEN 'In Progress'
                else 'boogy man'
            END RunStatus,
            --  jh.run_status,
            last_run_time =  (msdb.dbo.agent_datetime(run_date, run_time)),
                        last_end_time = (dateadd(ss,run_duration % 100 + ROUND((run_duration % 10000) / 100, 0, 0) * 60 + ROUND((run_duration % 1000000) / 10000, 0, 0) * 3600 ,msdb.dbo.agent_datetime(run_date, run_time)))
            --  ,j.enabled 
            
            ,
            CASE SJ.next_run_date
                        WHEN 0 THEN cast('n/a' as char(10))
                        ELSE convert(char(10), convert(datetime, convert(char(8),SJ.next_run_date)),120)  + ' ' + left(stuff((stuff((replicate('0', 6 - len(next_run_time)))+ convert(varchar(6),next_run_time),3,0,':')),6,0,':'),8)
                    END AS NextRunTime
            --           jh.duration
            FROM      msdb.dbo.sysjobs j
                    LEFT OUTER JOIN jobhistory jh
                    ON j.job_id = jh.job_id
                    join msdb..sysjobhistory sjh
                    on jh.instance_id = sjh.instance_id
                    left join msdb.dbo.sysjobschedules SJ on J.job_id = SJ.job_id  
            left join msdb.dbo.sysschedules SS on SS.schedule_id = SJ.schedule_id
            
            where sjh.run_status is not null
                and j.enabled = 1
            ORDER BY Runstatus,last_end_time desc 
        
        `" | Select-object * -ExcludeProperty RowError, RowState, Table, ItemArray, HasErrors | ConvertTo-Html | Out-String

        `$fragments += `"<br>$reportname</i>`"
        `$fragments += `$results 

        ConvertTo-HTML -Head `$head -Body `$fragments -PostContent `"<br><br><i>report generated: `$(Get-Date)</i> from $jobserver.`" | Out-File -FilePath `$path -Encoding ascii

        `$output = ConvertTo-HTML -Head `$head -Body `$fragments -PostContent `"<br><br><i>Report generated on `$(Get-Date)</i> via powershell on $jobserver.`" | Out-File -FilePath `$path -Encoding ascii

        #Define mail variables
        `$subject = `"$Reportname : `$(Get-Date -f MM-dd-yyyy)`" 

        Write-Host `"Sending email...`" -ForegroundColor Green
        [Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
        `$Parameters = @{
            FromAddress     = `"alertinbox@straightpathsql.com`"
            ToAddress       = `"$EmailRecipients`"
            CCAddress       = `"alertinbox@straightpathsql.com`"
            Subject         = `"$reportname for $Client_Name attached!`"
            Body            = `"Please find the $reportname for $Client_Name, attached!`"
            Token           =  Unprotect-CmsMessage -Path C:\temp\Demo\Sendgrid.txt
            FromName        = `"StraightPathSendGrid`"
            ToName          = `"David`"
        }
        Send-PSSendGridMail @Parameters
        "


    Write-Host "Creating XML File..." -ForegroundColor Green
    New-Item -Path "\\$jobserver\C$\Straightpath\Reports" -Name "$reportname`_Task.xml" -ItemType 'file' -value "<?xml version=`"1.0`" encoding=`"UTF-16`"?>
    <Task version=`"1.2`" xmlns=`"http://schemas.microsoft.com/windows/2004/02/mit/task`">
        <RegistrationInfo>
        <Date>2022-02-23T12:59:09.8081565</Date>
        <Author>David Seis, Straight Path IT Solutions, LLC.</Author>
        <URI>\$reportname</URI>
    </RegistrationInfo>
    <Triggers>
    <CalendarTrigger>
        <StartBoundary>2023-05-31T08:00:00</StartBoundary>
        <Enabled>true</Enabled>
        <ScheduleByDay>
        <DaysInterval>1</DaysInterval>
        </ScheduleByDay>
    </CalendarTrigger>
    <CalendarTrigger>
        <StartBoundary>2023-05-31T07:00:00</StartBoundary>
        <Enabled>true</Enabled>
        <ScheduleByDay>
        <DaysInterval>1</DaysInterval>
        </ScheduleByDay>
    </CalendarTrigger>
    <CalendarTrigger>
        <StartBoundary>2023-05-31T09:00:00</StartBoundary>
        <Enabled>true</Enabled>
        <ScheduleByDay>
        <DaysInterval>1</DaysInterval>
        </ScheduleByDay>
    </CalendarTrigger>
    </Triggers>
    <Principals>
        <Principal id=`"Author`">
        <UserId>$($ServiceAccount.GetNetworkCredential().username)</UserId>
        <LogonType>Password</LogonType>
        <RunLevel>HighestAvailable</RunLevel>
        </Principal>
    </Principals>
    <Settings>
        <MultipleInstancesPolicy>IgnoreNew</MultipleInstancesPolicy>
        <DisallowStartIfOnBatteries>true</DisallowStartIfOnBatteries>
        <StopIfGoingOnBatteries>true</StopIfGoingOnBatteries>
        <AllowHardTerminate>true</AllowHardTerminate>
        <StartWhenAvailable>false</StartWhenAvailable>
        <RunOnlyIfNetworkAvailable>false</RunOnlyIfNetworkAvailable>
        <IdleSettings>
        <StopOnIdleEnd>true</StopOnIdleEnd>
        <RestartOnIdle>false</RestartOnIdle>
        </IdleSettings>
        <AllowStartOnDemand>true</AllowStartOnDemand>
        <Enabled>true</Enabled>
        <Hidden>false</Hidden>
        <RunOnlyIfIdle>false</RunOnlyIfIdle>
        <WakeToRun>false</WakeToRun>
        <ExecutionTimeLimit>P3D</ExecutionTimeLimit>
        <Priority>7</Priority>
    </Settings>
    <Actions Context=`"Author`">
    <Exec>
        <Command>Powershell.exe</Command>
        <Arguments>-ExecutionPolicy Unrestricted -File `"C:\StraightPath\Reports\$reportname`_Script.ps1`"</Arguments>
    </Exec>
    </Actions>
    </Task>"

    Register-ScheduledTask -Xml (get-content "\\$jobserver\C$\StraightPath\Reports\$reportname`_Task.xml" | out-string) -Taskname "_REPORT_$reportname" -User ($ServiceAccount.GetNetworkCredential().username) -Password $($ServiceAccount.GetNetworkCredential().Password) 


. {
>> clear-host
>> 
>>     ## DEPENDENCIES
>>     # 1. Dbatools
>>     # 2. Service account access to all monitored servers
>> 
>>     <# Necessary Variables #>
>>     $reportname = "Job Run Report"
>> 
>>     $ClientName = 'Test-Lab' 
>>     $jobserver = 'Seis-Work'
>>     $monitoringTarget = $sqlinstance
>> 
>>     $ServiceAccount = $host.ui.PromptForCredential("Task Scheduler account", "Please enter the domainname\username and password for the service account that will run the tasks (password expiry = never is ideal). ", "DOMAIN\Account", "")
>> 
>>     $EmailRecipients = 'David.seis@straightpathsql.com;' #separate with semicolons
>> 
>>     Write-Host "Creating Powershell File..." -ForegroundColor Green
>>     New-Item -Path "\\$jobserver\C`$\StraightPath\Reports" -Name "$reportname`Script.ps1" -ItemType 'file' -value "
>>         
>>     <# Create C:\StraightPath\Reports for all the reports to go into#>
>>         `$dir = `"\\$jobserver\C$\StraightPath\Reports`"
>>         If(!(

`"UTF-16`"?>
>>     <Task version=`"1.2`" xmlns=`"http://schemas.microsoft.com/windows/2004/02/mit/task`">
>>         <RegistrationInfo>
>>         <Date>2022-02-23T12:59:09.8081565</Date>
>>         <Author>David Seis, Straight Path IT Solutions, LLC.</Author>
>>         <URI>\$reportname</URI>
>>     </RegistrationInfo>
>>     <Triggers>
>>     <CalendarTrigger>
>>         <StartBoundary>2023-05-31T08:00:00</StartBoundary>
>>         <Enabled>true</Enabled>
>>         <ScheduleByDay>
>>         <DaysInterval>1</DaysInterval>
>>         </ScheduleByDay>
>>     </CalendarTrigger>
>>     <CalendarTrigger>
>>         <StartBoundary>2023-05-31T07:00:00</StartBoundary>
>>         <Enabled>true</Enabled>
>>         <ScheduleByDay>
>>         <DaysInterval>1</DaysInterval>
>>         </ScheduleByDay>
>>     </CalendarTrigger>
>>     <CalendarTrigger>
>>         <StartBoundary>2023-05-31T09:00:00</StartBoundary>
>>         <Enabled>true</Enabled>
>>         <ScheduleByDay>
>>         <DaysInt

Creating Powershell File...




    Directory: \\Seis-Work\C$\StraightPath\Reports


Mode                 LastWriteTime         Length Name                                                                 
----                 -------------         ------ ----                                                                 
-a----          5/4/2024   1:24 AM           4559 Job Run ReportScript.ps1                                             
Creating XML File...


New-Item : The file '\\Seis-Work\C$\Straightpath\Reports\Job Run Report_Task.xml' already exists.
At line:100 char:5
+     New-Item -Path "\\$jobserver\C$\Straightpath\Reports" -Name "$rep ...
+     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : WriteError: (\\Seis-Work\C$\...Report_Task.xml:String) [New-Item], IOException
    + FullyQualifiedErrorId : NewItemIOError,Microsoft.PowerShell.Commands.NewItemCommand
 


Register-ScheduledTask : Access is denied.
At line:163 char:5
+     Register-ScheduledTask -Xml (get-content "\\$jobserver\C$\Straigh ...
+     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : PermissionDenied: (PS_ScheduledTask:Root/Microsoft/...S_ScheduledTask) [Register-Schedul 
   edTask], CimException
    + FullyQualifiedErrorId : HRESULT 0x80070005,Register-ScheduledTask
 




## 3.4 Advanced PowerShell Automation: Loading

Using powershell to load data collected from a variety of sources simplifies management of the data being collected. Seting up scheudled tasks to extract the data, make any needed modifications, and then load it into a management database is the primary reason I needed to develop the advanced skills I am sharing here.

In [26]:
<# code to create a management table to load the previously colelcted data into #>

<# 004 - Loading Data #>
$managementInstance = 'seis-work'
$managementDatabase = "DB_administration"

Write-Host "Create Tracking Table ..." -ForegroundColor Green
    Invoke-dbaquery -SqlInstance $managementInstance  -Query "
    USE $managementDatabase
    CREATE TABLE InstanceTable (
    Inst_ID INT IDENTITY(1,1) NOT NULL
    , 	machine_name NVARCHAR(256)
    ,   Instance_name NVARCHAR(256)
    ,	Env_Type NVARCHAR(30) DEFAULT ''
    ,	SQL_Version NVARCHAR(10)
    ,	SQL_Edition NVARCHAR(50)
    ,	Product_Level NVARCHAR(20)
    , 	Update_Level NVARCHAR(20)
    ,	SQL_Build NVARCHAR(20)
    ,	Ola_Version NVARCHAR(20)
    ,	First_Responder_Version NVARCHAR(20)
    , 	SQL_Account NVARCHAR(255)
    , 	SQL_Agent_Account NVARCHAR(255)
    ,	Uptime_Days SMALLINT
    , 	is_sa_disabled BIT
    ,	local_net_address VARCHAR(48)
    ,	local_tcp_port INT
    ,	num_error_logs TINYINT
    ,   Collection_date datetime
    ,	[Valid_From] [datetime2](7) GENERATED ALWAYS AS ROW START NOT NULL
    ,	[Valid_To] [datetime2](7) GENERATED ALWAYS AS ROW END NOT NULL
    ,	PERIOD FOR SYSTEM_TIME (Valid_From, Valid_To)
    ,	CONSTRAINT [PK_InstanceTable_T] PRIMARY KEY CLUSTERED 
    ([Inst_ID] ASC)
    WITH 
    (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY],) ON [PRIMARY]
    WITH
    (SYSTEM_VERSIONING = ON 
    (HISTORY_TABLE = [dbo].[InstanceTable_T_history]
    ,	HISTORY_RETENTION_PERIOD = 12 MONTHS)
    )
GO

    CREATE UNIQUE NONCLUSTERED INDEX IX_InstanceTable_machinename_instancename_localnetaddress
    ON InstanceTable (machine_name, Instance_name,local_net_address)

    CREATE TABLE ToolsAudit (
    Tool_Name NVARCHAR(50) PRIMARY KEY 
    ,	Supported_Version NVARCHAR(30) 
    )
    INSERT ToolsAudit (Tool_Name, Supported_Version)
    VALUES 	('Ola_Hallengrin', '2022-12-03')
    ,	('First_Responder_Toolkit', '8.18')
    GO
    
    CREATE TABLE SQLBuilds (
        Version NVARCHAR(50) PRIMARY KEY 
        ,	SupportedBuild NVARCHAR(30) 
        )
        INSERT SQLBuilds (Version, SupportedBuild)
        VALUES ('2022', '16.0.4120.1')
        ,	('2019', '15.0.4365.2')
        ,	('2017', '14.0.3465.1')
        /*Azure connect updates only Below*/
        ,	('2016', '13.0.6435.1')
        ,	('2014', '12.0.6449.1')
        ,	('2012', '11.0.7507.2')
        ,	('2008 R2', '10.50.6560.0')
        ,	('2008', '10.0.6814.4')
    GO

"


Create Tracking Table ...


In [27]:
<# code to load the staged data #>

$Instancefiles = Get-ChildItem -Path $outputPath\* | Where-Object Name -like "*InstanceTracking*"
foreach ($inst in $Instancefiles) {
Import-DbaCsv -Path $inst -SqlInstance $managementInstance  -database $managementdatabase  -schema dbo -AutoCreateTable -Delimiter ',' -enableexception 

Invoke-DbaQuery -SqlInstance $managementInstance  -database $database  -Query "
USE $managementdatabase

UPDATE InstanceTable
SET 
SQL_Version = D.SQL_Version
, SQL_Edition = D.SQL_Edition
, Product_Level = D.Product_Level
, Update_Level = D.Update_Level
, SQL_Build = D.SQL_Build
, Ola_Version = D.Ola_Version
, First_Responder_Version = D.First_Responder_Version
, SQL_Account = D.SQL_Account 
, SQL_Agent_Account = D.SQL_Agent_Account 
, Uptime_Days = D.Uptime_Days 
, is_sa_disabled = D.is_sa_disabled
, local_net_address = D.local_net_address 
, local_tcp_port = D.local_tcp_port
, num_error_logs = D.num_error_logs
, collection_date = D.collection_date
FROM InstanceTable I
JOIN [dbo].[$((get-item $inst).basename)] D 
ON D.Machine_Name = I.Machine_Name 
AND D.instance_Name = I.Instance_Name

INSERT INTO InstanceTable (Machine_name,Instance_name, SQL_Version, SQL_Edition, Product_Level, Update_Level, SQL_Build, Ola_Version, First_Responder_Version, SQL_Account, SQL_Agent_Account, Uptime_Days, is_sa_disabled, local_net_address, local_tcp_port, num_error_logs,collection_date)
SELECT DISTINCT
Machine_Name
, instance_name
, SQL_Version
, SQL_Edition
, Product_Level
, Update_Level
, SQL_Build
, Ola_Version
, First_Responder_Version
, SQL_Account
, SQL_Agent_Account
, Uptime_Days
, is_sa_disabled
, local_net_address 
, local_tcp_port
, num_error_logs
, collection_date
FROM [dbo].[$((get-item $inst).basename)] D
WHERE D.Instance_name NOT IN (SELECT Instance_name From InstanceTable)
    AND D.machine_name NOT IN (SELECT Machine_name from InstanceTable)

DROP TABLE [dbo].[$((get-item $inst).basename)]
"
}



ComputerName  : Seis-WORK
InstanceName  : MSSQLSERVER
SqlInstance   : Seis-WORK
Database      : DB_administration
Table         : Home-lab_InstanceTracking
Schema        : dbo
RowsCopied    : 1
Elapsed       : 120.34 ms
RowsPerSecond : 8.4
Path          : C:\temp\Demo\Home-lab_InstanceTracking.csv



In [29]:
<# query to get action items from the data #>

Invoke-DbaQuery -SqlInstance $managementInstance -Query "
USE $managementdatabase
SELECT 
machine_name
,	Instance_name
,   CASE 
        WHEN i.First_Responder_Version <> (SELECT Supported_Version FROM ToolsAudit AS ToolsAudit_1 WHERE (Tool_Name LIKE '%first%')) 
            THEN 'YES || Current: [' + i.First_Responder_Version + '] =/= Target: [' + (SELECT     Supported_Version FROM        ToolsAudit AS ToolsAudit_1 WHERE (Tool_Name LIKE '%first%')) + ']' 
        WHEN i.First_Responder_Version IS NULL 
            THEN 'YES || Rollout Needed, First Responder not present.' 
        ELSE 'NO  || [' + i.First_Responder_Version + ']' 
    END AS first_responder_update_needed
,	CASE 
        WHEN i.Ola_Version <> (SELECT Supported_Version FROM ToolsAudit AS ToolsAudit_1 WHERE (Tool_Name LIKE '%Ola%')) 
            THEN 'YES || Current: [' + i.Ola_Version + '] =/= Target: [' + (SELECT Supported_Version FROM ToolsAudit WHERE (Tool_Name LIKE '%Ola%')) + ']' 
        WHEN i.Ola_Version IS NULL 
            THEN 'YES || Rollout Needed, Ola not present.' 
        ELSE 'NO  || [' + i.Ola_Version + ']' 
    END AS [Ola_Update_Needed]
,	CASE 
        WHEN i.SQL_Build <> s.SupportedBuild 
            THEN 'YES || Current: [' + i.SQL_Build + '] =/= Target: [' + s.SupportedBuild + ']' 
        ELSE 'NO  || Current: [' + i.SQL_Build + ']' 
    END AS [Patch_Needed]
FROM InstanceTable i
JOIN SQLBuilds AS s
    ON i.SQL_Version = s.Version
GO

" 



machine_name                  : Seis-WORK
Instance_name                 : Seis-WORK
first_responder_update_needed : YES || Current: [8.13] =/= Target: [8.18]
Ola_Update_Needed             : NO  || [2022-12-03]
Patch_Needed                  : YES || Current: [16.0.1115.1] =/= Target: [16.0.4120.1]





## 3.5 Advanced PowerShell Automation: Tooling

Using powershell to simplify repetitive but important tasks such as patching, keeping tools up to date and making sure systems are meeting internal compliance standards is a time saving effort. There is some time to building and testing these scripts, but the time dividends are remarkable.

In [30]:
<# SSMS update script #>
    
    <# Current install audit #>
    Get-WmiObject Win32_product | Where-Object {$_.name -eq "SQL Server Management Studio"} | ForEach-object {
        Write-Host "SSMS: Currently Installed Versions of SSMS [Version: $($_.Version)]`..." -ForegroundColor Green
    }


    <# New Installer download #>
    $ssmsUrl = "https://aka.ms/ssmsfullsetup"
    $destination = "C:\Temp\ssms_installer.exe"

    try {
        Write-Host "SSMS: Downloading newest SSMS Installer..." -ForegroundColor Green
        $WebClient = New-Object System.Net.WebClient
        $webclient.DownloadFile($ssmsUrl, $destination)
    }
    catch {
        Write-Host "Unable to download the latest SSMS installer, please download and install manually." -ForegroundColor Red -BackgroundColor White
        return
    }

    <# Installer was successfully downloaded, uninstall old ssms versions #>
    Write-Host "SSMS: SSMS Update flag was on, enumerating SSMS installs..." -ForegroundColor Green
        Invoke-Command -ScriptBlock {
            Get-WmiObject Win32_product | Where-Object {$_.name -eq "SQL Server Management Studio"} | ForEach-object {
                Write-Host "SSMS: Uninstalling SSMS [Version: $($_.Version)]`..." -ForegroundColor Green
                $_.Uninstall() | Out-null
            }
        }


    # Install SSMS silently
    $install_path = "`"C:\Program Files (x86)\Microsoft SQL Server Management Studio 19`""
    $params = " /Install /Passive SSMSInstallRoot=$install_path /quiet"
            
    Write-Host "SSMS: Installing SSMS..." -ForegroundColor Green
    Start-Process -FilePath $destination -ArgumentList $params -Wait
    Remove-Item $destination
    #Write-Host "SSMS: Server restart is required to implement changes fully" -ForegroundColor DarkRed
    Invoke-Command -ScriptBlock {
        Get-WmiObject Win32_product | Where-Object {$_.name -eq "SQL Server Management Studio"} | ForEach-object {
            Write-Host "SSMS: Currently Installed Versions of SSMS [Version: $($_.Version)]`..." -ForegroundColor Green
        }
    }

. {
>> <# SSMS update script - pwoershell admin required#>
>>     
>>     <# Current install audit #>
>>     Get-WmiObject Win32_product | Where-Object {$_.name -eq "SQL Server Management Studio"} | ForEach-object {
>>         Write-Host "SSMS: Currently Installed Versions of SSMS [Version: $($_.Version)]`..." -ForegroundColor Green
>>     }
>> 


>> 
>>     <# New Installer download #>
>>     $ssmsUrl = "https://aka.ms/ssmsfullsetup"
>>     $destination = "C:\Temp\ssms_installer.exe"
>> 
>>     try {
>>         Write-Host "SSMS: Downloading newest SSMS Installer..." -ForegroundColor Green
>>         $WebClient = New-Object System.Net.WebClient
>>         $webclient.DownloadFile($ssmsUrl, $destination)
>>     }
>>     catch {
>>         Write-Host "Unable to download the latest SSMS installer, please download and install manually." -ForegroundColor Red -BackgroundColor White
>>         return
>>     }
>> 
>>     <# Installer was successfully downloaded, uninstall old ssms versions #>
>>     Write-Host "SSMS: SSMS Update flag was on, enumerating SSMS installs..." -ForegroundColor Green
>>         Invoke-Command -ScriptBlock {
>>             Get-WmiObject Win32_product | Where-Object {$_.name -eq "SQL Server Management Studio"} | ForEach-object {
>>                 Write-Host "SSMS: Uninstalling SSMS [Version: $($_.Version)]`..." 

SSMS: Currently Installed Versions of SSMS [Version: 15.0.18424.0]...


SSMS: Currently Installed Versions of SSMS [Version: 15.0.18424.0]...
SSMS: Downloading newest SSMS Installer...


SSMS: SSMS Update flag was on, enumerating SSMS installs...


SSMS: Uninstalling SSMS [Version: 15.0.18424.0]...


SSMS: Uninstalling SSMS [Version: 15.0.18424.0]...


SSMS: Installing SSMS...


SSMS: Currently Installed Versions of SSMS [Version: 15.0.18424.0]...


SSMS: Currently Installed Versions of SSMS [Version: 15.0.18424.0]...
SSMS: Currently Installed Versions of SSMS [Version: 20.1.10.0]...


In [5]:
<# Environment Cleanup #>
Remove-Item -Path $CSVpath 

If((test-path -PathType container $outputPath)) 
{ 
    remove-Item -Path $outputPath
    Write-Host "$outputPath Directory was removed"
}

Write-Host "Environment cleaned"